In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
from pathlib import Path
import re
import requests
from io import StringIO
import random
import pdb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Upload trips data
trips = pd.read_csv('/content/drive/MyDrive/Code/Results/Gipuzkoa_MT2_extended_train.csv',index_col=0)
#Filtrar para interes
eliminar = ['Prov_Ori', 'Prov_Des', 'Com_Ori', 'Com_Des',
       'Act_Ori', 'Act_Des', 'Proposito', 'Modo', 'Hora_Ini', 'Dur_Tot',
       'Recur', 'Bill', 'Park', 'Peaje', 'Provincia', 'Municipio', 'Comarca',
       'Per_hog', 'Turismos', 'Motos', 'Tipo_familia', 'Sexo', 'Edad',
       'Actividad', 'crnt_tur', 'crnt_mot', 'crnt_otr', 'Income']
trips = trips.drop(columns=eliminar)
trips

,Mun_Ori,Mun_Des,O_long,O_lat,D_long,D_lat
0,Ordizia,Lasarte-Oria,-2.185569,43.053250,-2.022158,43.270200
1,Irun,Zarautz,-1.788292,43.347005,-2.178315,43.285188
2,Donostia/San Sebastian,Pasaia,-1.984329,43.319607,-1.917421,43.316461
3,Oñati,Arrasate/Mondragon,-2.430788,43.049372,-2.489183,43.065417
4,Donostia/San Sebastian,Zumaia,-1.963417,43.302974,-2.259040,43.298208
...,...,...,...,...,...,...
150487,Errenteria,Hondarribia,-1.900690,43.314035,-1.827935,43.341005
150488,Hondarribia,Donostia/San Sebastian,-1.797642,43.361603,-1.981600,43.305572
150489,Tolosa,Beasain,-2.078414,43.130016,-2.236591,43.045380
150490,Donostia/San Sebastian,Lezo,-1.963862,43.316341,-1.894946,43.320786


In [ ]:
# Añadir lat long de ciudad origen y destino
towns = pd.read_excel('/content/drive/MyDrive/Code/Data/pueblos_gipuzkoa.xlsx')
# Filtrar
eliminar = ['Region','Comarca',
       'Altitud (m.s.n.m.)', 'Superficie (kmÂ²)', 'PoblaciÃ³n (2019)',
       'Densidad (hab./kmÂ²)', 'Código', 'Incluido']
towns = towns.drop(columns=eliminar)
towns.rename(columns={'Latitud': 'Lat_pueblo'}, inplace=True)
towns.rename(columns={'Longitud': 'Long_pueblo'}, inplace=True)
towns

,Town,Lat_pueblo,Long_pueblo
0,Abaltzisketa,43.051156,-2.110379
1,Aduna,43.214811,-2.054751
2,Aia,43.245113,-2.152427
3,Aizarnazabal,43.265408,-2.232831
4,Albiztur,43.128648,-2.143870
...,...,...,...
83,Zerain,43.010677,-2.284802
84,Zestoa,43.241819,-2.237987
85,Zizurkil,43.213223,-2.077485
86,Zumaia,43.287645,-2.246113


In [ ]:
# Merge

trips = pd.merge(trips, towns, left_on='Mun_Ori', right_on='Town')
trips.rename(columns={'Lat_pueblo': 'Lat_pueblo_O'}, inplace=True)
trips.rename(columns={'Long_pueblo': 'Long_pueblo_O'}, inplace=True)

trips = pd.merge(trips, towns, left_on='Mun_Des', right_on='Town')
trips.rename(columns={'Lat_pueblo': 'Lat_pueblo_D'}, inplace=True)
trips.rename(columns={'Long_pueblo': 'Long_pueblo_D'}, inplace=True)

trips = trips.drop(columns=['Town_x', 'Town_y'])
trips

,Mun_Ori,Mun_Des,O_long,O_lat,D_long,D_lat,Lat_pueblo_O,Long_pueblo_O,Lat_pueblo_D,Long_pueblo_D
0,Ordizia,Lasarte-Oria,-2.185569,43.053250,-2.022158,43.270200,43.063758,-2.181832,43.263787,-2.011931
1,Ordizia,Lasarte-Oria,-2.185310,43.046085,-2.020837,43.271231,43.063758,-2.181832,43.263787,-2.011931
2,Ordizia,Lasarte-Oria,-2.166916,43.055082,-2.021995,43.268590,43.063758,-2.181832,43.263787,-2.011931
3,Ordizia,Lasarte-Oria,-2.176754,43.054089,-2.003686,43.260811,43.063758,-2.181832,43.263787,-2.011931
4,Ordizia,Lasarte-Oria,-2.178086,43.053536,-2.018444,43.270202,43.063758,-2.181832,43.263787,-2.011931
...,...,...,...,...,...,...,...,...,...,...
150487,Arrasate/Mondragon,Aretxabaleta,-2.492852,43.065881,-2.505237,43.034965,43.074523,-2.484149,43.030186,-2.485701
150488,Arrasate/Mondragon,Aretxabaleta,-2.493563,43.065330,-2.504162,43.029494,43.074523,-2.484149,43.030186,-2.485701
150489,Arrasate/Mondragon,Aretxabaleta,-2.500356,43.060983,-2.504162,43.029494,43.074523,-2.484149,43.030186,-2.485701
150490,Arrasate/Mondragon,Aretxabaleta,-2.489647,43.065882,-2.509114,43.028928,43.074523,-2.484149,43.030186,-2.485701


In [ ]:
trips.to_csv('/content/drive/MyDrive/Code/Results/exparcir_trips_kepler.csv')

In [ ]:
# Filtra las filas que cumplan con tus condiciones
condiciones = ((trips['Mun_Ori'] == 'Eibar') & (trips['Mun_Des'] == 'Donostia/San Sebastian')) | \
              ((trips['Mun_Ori'] == 'Bergara') & (trips['Mun_Des'] == 'Beasain')) | \
              ((trips['Mun_Ori'] == 'Ordizia') & (trips['Mun_Des'] == 'Lasarte-Oria'))

trips_especial = trips[condiciones]
trips_especial.to_csv('/content/drive/MyDrive/Code/Results/exparcir_trips_kepler_especial.csv')
trips_especial

,Mun_Ori,Mun_Des,O_long,O_lat,D_long,D_lat,Lat_pueblo_O,Long_pueblo_O,Lat_pueblo_D,Long_pueblo_D
0,Ordizia,Lasarte-Oria,-2.185569,43.053250,-2.022158,43.270200,43.063758,-2.181832,43.263787,-2.011931
1,Ordizia,Lasarte-Oria,-2.185310,43.046085,-2.020837,43.271231,43.063758,-2.181832,43.263787,-2.011931
2,Ordizia,Lasarte-Oria,-2.166916,43.055082,-2.021995,43.268590,43.063758,-2.181832,43.263787,-2.011931
3,Ordizia,Lasarte-Oria,-2.176754,43.054089,-2.003686,43.260811,43.063758,-2.181832,43.263787,-2.011931
4,Ordizia,Lasarte-Oria,-2.178086,43.053536,-2.018444,43.270202,43.063758,-2.181832,43.263787,-2.011931
...,...,...,...,...,...,...,...,...,...,...
52890,Bergara,Beasain,-2.412930,43.116293,-2.188592,43.051942,43.122005,-2.414838,43.073261,-2.217907
52891,Bergara,Beasain,-2.416421,43.119318,-2.220499,43.049897,43.122005,-2.414838,43.073261,-2.217907
52892,Bergara,Beasain,-2.413542,43.120035,-2.217735,43.049998,43.122005,-2.414838,43.073261,-2.217907
52893,Bergara,Beasain,-2.416974,43.108948,-2.233440,43.052456,43.122005,-2.414838,43.073261,-2.217907
